# Convert to yoloformat

In [14]:
import json
import os
from pathlib import Path
from tqdm import tqdm

# Load the JSON data
data_path = '/Users/apple/Desktop/test/polygon/drivable_val.json'
with open(data_path, 'r') as file:
    data = json.load(file)

output_dir = Path('/Users/apple/Desktop/test/polygon/polygon_label1')
output_dir.mkdir(parents=True, exist_ok=True)  # Ensure directory exists

def normalize_coordinates(vertices, image_width, image_height):
    """
    Normalize the polygon vertex coordinates.
    """
    normalized_vertices = []
    for x, y in vertices:
        norm_x = x / image_width
        norm_y = y / image_height
        normalized_vertices.extend([norm_x, norm_y])
    return normalized_vertices

def process_entry(entry):
    image_name = entry.get('name')
    if not image_name:
        print("Warning: Missing image name.")
        return
    
    image_width = entry.get('width', 1280)  # Provide default width if missing
    image_height = entry.get('height', 720)  # Provide default height if missing
    output_file_path = output_dir / f'{image_name.replace(".jpg", ".txt")}'

    with open(output_file_path, 'w+') as output_file:
        labels = entry.get('labels', [])
        for label in labels:
            category = label.get('category')
            if category is None:
                print(f"Warning: Missing category in image {image_name}")
                continue
            category_index = label.get('id')  # Replace this with actual mapping to category indices
            
            poly2d = label.get('poly2d', [])
            for polygon in poly2d:
                vertices = polygon.get('vertices')
                if not vertices:
                    print(f"Warning: Missing vertices in image {image_name}, category {category}")
                    continue
                
                normalized_vertices = normalize_coordinates(vertices, image_width, image_height)
                line = f"{category_index} " + " ".join(map(str, normalized_vertices))
                output_file.write(line + '\n')

# Process each entry in the loaded data
for entry in tqdm(data):
    process_entry(entry)


100%|██████████| 10000/10000 [00:00<00:00, 12944.50it/s]


# Dataset strcuture construction

In [15]:
from pathlib import Path
import os
import shutil
import random
from sklearn.model_selection import train_test_split

# Directories for images and labels
image_dir = Path('/Users/apple/Desktop/Sensing System/StarkVision/bdd100k_new/val')
label_dir = Path('/Users/apple/Desktop/test/polygon/polygon_label1')

# Output base directory
base_dir = Path('/Users/apple/Desktop/test/polygon/drivable_test1')

# Create directories for train, validation, and test
train_dir = base_dir / 'train'
val_dir = base_dir / 'val'
test_dir = base_dir / 'test'

# Subdirectories for images and labels
for path in [train_dir, val_dir, test_dir]:
    for subpath in ['images', 'labels']:
        (path / subpath).mkdir(parents=True, exist_ok=True)


randomly select 100

In [16]:
# List all image files and check corresponding labels
all_images = [f.stem for f in image_dir.glob('*.jpg')]  # List images by filename stem (no extension)
all_labels = [f.stem for f in label_dir.glob('*.txt')]  # List labels similarly

# Filter images to those that have a corresponding label
valid_images = [image for image in all_images if image in all_labels]

# Randomly select 1000 images from those that have labels
selected_images = random.sample(valid_images, 100) if len(valid_images) >= 100 else valid_images

# Split data
train_files, test_files = train_test_split(selected_images, test_size=0.2, random_state=1)
train_files, val_files = train_test_split(train_files, test_size=0.25, random_state=1)


In [17]:
def move_files(files, source_dir, target_dir, file_type):
    """
    Move files from the source directory to the target directory.
    `file_type` can be 'images' or 'labels' with corresponding extensions.
    """
    ext = '.jpg' if file_type == 'images' else '.txt'
    for file in files:
        src_path = source_dir / f"{file}{ext}"
        tgt_path = target_dir / file_type / f"{file}{ext}"
        shutil.move(src_path, tgt_path)

# Move image and label files
move_files(train_files, image_dir, train_dir, 'images')
move_files(train_files, label_dir, train_dir, 'labels')
move_files(val_files, image_dir, val_dir, 'images')
move_files(val_files, label_dir, val_dir, 'labels')
move_files(test_files, image_dir, test_dir, 'images')
move_files(test_files, label_dir, test_dir, 'labels')